# Deep Learning - Tutorial Facial Keypoints Detection
### Magister en Data Science

#### Instituto Data Science, Universidad del Desarrollo

By Hernan Rivera 

* Tutorial esta basado en el tutorial de Daniel Nouri fundadod de Natural Vision. 
* Original: http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial

Tutorial Contents

* Requisitos previos
* Los datos
* Primer modelo: una sola capa oculta.
* Probándolo
* Segundo modelo: convolutions.
* Aumento de datos
* Cambio de la tasa de aprendizaje y el impulso con el tiempo
* Abandonar
* Especialistas en formación.
* Pre-entrenamiento supervisado.
* Conclusión

### 1. Requisitos previos

* Se debe instalar las librerias Theano y lasagne

In [1]:
#!pip install -r https://raw.githubusercontent.com/dnouri/kfkd-tutorial/master/requirements.txt
# En mi caso previamente instalada, por lo tanto, se comenta el comando.

### 2. Los datos

* Cremoa funcion para chequear los datos dispoibles para el desafio

In [2]:
import os
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


* Los datos se descargan desde el sitio web de kagle (https://www.kaggle.com/c/facial-keypoints-detection/data)

In [3]:
FTRAIN = './data/training.zip'
FTEST = './data/test.zip'

In [4]:
def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

In [5]:
X, y = load()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

* Podemos observar las caractetisticas principales del dataset, en donde destaca lo siguiente:
 > 
 * La data de entrenamineto consta de 7049 imagenes de 96 x 96 pixeles en escala de grises.
 * Existen 30 caracteristicas que corresponden a los pares (puntos) X, Y de los 15 puntos claves de la cara.
 * Para algunos puntos claves solo tenemso aprox 2000 etiquetas, mientras que para otros cerca de 7000 etiquetas.
 * **y.shape == (2140, 30)** nos dice que solo hay 2140 imagenes en el dataset con todas las features
 * Las imagenes estan en escala [0,1] en vez de [0, 255]
 * Los valores de X, Y estan entre [-1,1] y las imagenes de [0,95]

### 3. Primer modelo: una sola capa oculta.

* Creamos una red Lasagne con una solo una capa:
 > 
 * Primero inicializamos la red con 3 capas (input, hidden y output). En este punto se especifica el nombre y su orden.
 * Luego se especifica las caracteristicas de cada capa
 * Al establecer output_nonlinearity como None se utilza la funcion de activacion por defecto para capas densas, esto es rectifier, que corresponde a un simple max(0, x).
 
<img src="img/rectifier.png" />
  
  * Los pesos son inicializados por Lasagne utilizado una distribución uniforme utilizando inicialización <a href="http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf">Glorot-style</a>
  * Se utilizará a "nesterov_momentum gradient descendent" como metodo de optimización
  * El Learning rate será de 0.01
  * Se estable como un problema de regresión
  * Se establece el numero de epoch a 400

<img src="img/methods.gif" />

* Comparación de metodos de optimización(by Alec Radford). La estrella denota el minimo global en el plano de errores. Nosotros usaremos Nesterov's Accelerated Gradient Descent (NAG) en este tutorial.

In [6]:
net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1, # Print 1, Not print 0
    )

In [7]:
net1.fit(X, y)

# Neural Network with 924730 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input     9216
  1  hidden     100
  2  output      30

  epoch    trn loss    val loss    trn/val  dur
-------  ----------  ----------  ---------  -----
      1     0.13361     0.03748    3.56498  0.64s
      2     0.02498     0.01717    1.45463  0.32s
      3     0.01519     0.01475    1.02916  0.28s
      4     0.01377     0.01353    1.01758  0.29s
      5     0.01272     0.01269    1.00272  0.27s
      6     0.01201     0.01210    0.99278  0.29s
      7     0.01149     0.01164    0.98710  0.28s
      8     0.01106     0.01125    0.98297  0.30s
      9     0.01069     0.01092    0.97932  0.28s
     10     0.01037     0.01062    0.97631  0.29s
     11     0.01009     0.01036    0.97380  0.28s
     12     0.00984     0.01012    0.97153  0.28s
     13     0.00961     0.00991    0.96944  0.28s
     14     0.00940     0.00972    0.96747  0.29s
     15     0.00921     0.00

    118     0.00438     0.00516    0.84712  0.48s
    119     0.00436     0.00515    0.84617  0.40s
    120     0.00434     0.00513    0.84529  0.41s
    121     0.00432     0.00512    0.84436  0.30s
    122     0.00430     0.00510    0.84344  0.27s
    123     0.00428     0.00508    0.84256  0.27s
    124     0.00427     0.00507    0.84167  0.28s
    125     0.00425     0.00505    0.84081  0.29s
    126     0.00423     0.00504    0.83990  0.28s
    127     0.00421     0.00502    0.83904  0.30s
    128     0.00420     0.00501    0.83814  0.30s
    129     0.00418     0.00499    0.83727  0.31s
    130     0.00416     0.00498    0.83638  0.30s
    131     0.00415     0.00496    0.83552  0.32s
    132     0.00413     0.00495    0.83465  0.29s
    133     0.00411     0.00493    0.83405  0.29s
    134     0.00410     0.00492    0.83316  0.30s
    135     0.00408     0.00490    0.83230  0.31s
    136     0.00406     0.00489    0.83161  0.30s
    137     0.00405     0.00487    0.83073  0.30s


    239     0.00292     0.00385    0.75803  0.26s
    240     0.00291     0.00384    0.75774  0.25s
    241     0.00291     0.00384    0.75736  0.25s
    242     0.00290     0.00383    0.75652  0.26s
    243     0.00289     0.00383    0.75585  0.25s
    244     0.00288     0.00382    0.75563  0.25s
    245     0.00288     0.00381    0.75488  0.25s
    246     0.00287     0.00380    0.75440  0.25s
    247     0.00286     0.00380    0.75343  0.27s
    248     0.00286     0.00379    0.75311  0.31s
    249     0.00285     0.00379    0.75265  0.26s
    250     0.00284     0.00378    0.75183  0.25s
    251     0.00284     0.00377    0.75143  0.25s
    252     0.00283     0.00377    0.75063  0.26s
    253     0.00282     0.00376    0.75011  0.26s
    254     0.00282     0.00376    0.74965  0.36s
    255     0.00281     0.00375    0.74935  0.37s
    256     0.00280     0.00375    0.74854  0.37s
    257     0.00280     0.00374    0.74771  0.27s
    258     0.00279     0.00373    0.74756  0.26s


    366     0.00232     0.00333    0.69610  0.38s
    367     0.00232     0.00332    0.69767  0.35s
    368     0.00231     0.00330    0.69971  0.28s
    369     0.00231     0.00329    0.70174  0.28s
    370     0.00231     0.00328    0.70416  0.28s
    371     0.00232     0.00328    0.70698  0.27s
    372     0.00232     0.00328    0.70811  0.28s
    373     0.00230     0.00328    0.70069  0.29s
    374     0.00229     0.00336    0.68236  0.31s
    375     0.00236     0.00356    0.66110  0.27s
    376     0.00249     0.00377    0.66109  0.30s
    377     0.00257     0.00374    0.68691  0.29s
    378     0.00247     0.00345    0.71617  0.28s
    379     0.00230     0.00326    0.70577  0.26s
    380     0.00241     0.00359    0.67107  0.26s
    381     0.00289     0.00409    0.70756  0.26s
    382     0.00311     0.00384    0.81110  0.25s
    383     0.00266     0.00330    0.80605  0.25s
    384     0.00229     0.00345    0.66390  0.25s
    385     0.00251     0.00408    0.61532  0.26s


NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x10fcbe0b8>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x10fcbe080>,
     check_input=True, custom_scores=None, hidden_num_units=100,
     input_shape=(None, 9216),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=400, more_params={},
     objective=<function objective at 0x10fea31e0>,
     objective_loss_function=<function squared_error at 0x10fbd21e0>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog object at 0x10fee1470>],
     on_training_finished=[],
     on_training_started=[<nolearn.lasagne.handlers.PrintLayerInfo object at 0x10fee14a8>],
     output_nonlinearity=None, output_num_units=30, regression=True,
     scores_train=[], scores_valid=[],
     train_spl

* En las lineas de ejecución que corresponde a los epoch se puede observar la perdida (MSE) para el set de entrenamiento y el set de validación y el ratio entre ellos (Train / Val). NeuralNet por defecto divide la data para establecer un 20% para evaluacion
* Que tan buenos fueron los resultados? ** Train Loss =  0.00321 **
* Recuerden que dividimos el objetivo (target) por 48, cuando los escalamos para que fuera un intervalo [-1, 1]. Por lo tanto, para calcular el Root-Mean-Squart-Error, debemos miltiplicar por 48. 

In [8]:
np.sqrt(0.00321) * 48

2.7195293710493367